chexnet: horizontal flipping
yao: random rotations from -15 to 15 degrees
    scaled between 80 and 120%
    translation in 4 directions by 25 pixels
baltruschat: random rotations by +- 7 degrees
    horizontal flipping
    aspect ration variation between 3:4 and 4:3
    patch sampling between 8 and 100% of the original image
    weiss ja nicht: rescaling to 256 / 480 and then center crop
gündel: horizontal flipping
    cropping 448 out of 512: per corner and middle
shen: flipping
    adjusting brightness and contrast
shamrat: rotate up to 90 degrees
    translate
    vertical flip